<a href="https://colab.research.google.com/github/aseftian/PyColab/blob/main/MJ_Image_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pillow requests python-dotenv asyncio

In [ ]:
import pathlib
import csv
import textwrap
from IPython.display import display, Markdown
from PIL import Image
from google.colab import drive
from google.colab import files
import os
import functools
import typing
import asyncio
import requests
import time
from os.path import basename

# Function to convert text to markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Mount Google Drive
drive.mount('/content/drive')

# Define image folder path
image_folder_path = '/content/drive/MyDrive/0'

save_as_format = 'PNG'
optimize_image_size = True # fill with True or False

input_list_image = pathlib.Path('/content/drive/MyDrive/0/list_url_image.txt')

# Define output CSV path
output_csv_path = pathlib.Path('/content/drive/MyDrive/0/image_metadata.csv')

# fungsi untuk memisahkan image yang di download
def split_image(image_file):
    with Image.open(image_file) as im:
        # Get the width and height of the original image
        width, height = im.size
        # Calculate the middle points along the horizontal and vertical axes
        mid_x = width // 2
        mid_y = height // 2
        # Split image into four equal parts
        top_left = im.crop((0, 0, mid_x, mid_y))
        top_right = im.crop((mid_x, 0, width, mid_y))
        bottom_left = im.crop((0, mid_y, mid_x, height))
        bottom_right = im.crop((mid_x, mid_y, width, height))

        return top_left, top_right, bottom_left, bottom_right

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:

        # Define the input and output folder path
        input_folder = f"{image_folder_path}/input"
        output_folder = f"{image_folder_path}/output"

        # Check if the output folder exists, and create it if necessary
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Check if the input folder exists, and create it if necessary
        if not os.path.exists(input_folder):
            os.makedirs(input_folder)

        arq_name = basename(response.url)
        filename, tsh = arq_name.split("?", 1)

        with open(f"{input_folder}/{filename}", "wb") as f:
            f.write(response.content)
        f.close()

        input_file = os.path.join(input_folder, filename)

        file_basename = os.path.splitext(filename)[0]
        top_left, top_right, bottom_left, bottom_right = split_image(input_file)


        if save_as_format in ('JPG','JPEG'):
            file_ext = '.jpg'
            top_left.save(os.path.join(output_folder, f"{file_basename}_1_top_left{file_ext}"), "JPEG", optimize=optimize_image_size, quality=100)
            top_right.save(os.path.join(output_folder, f"{file_basename}_2_top_right{file_ext}"), "JPEG", optimize=optimize_image_size, quality=100)
            bottom_left.save(os.path.join(output_folder, f"{file_basename}_3_bottom_left{file_ext}"), "JPEG", optimize=optimize_image_size, quality=100)
            bottom_right.save(os.path.join(output_folder, f"{file_basename}_4_bottom_right{file_ext}"), "JPEG", optimize=optimize_image_size, quality=100)
        else:
            file_ext = '.png'
            top_left.save(os.path.join(output_folder, f"{file_basename}_1_top_left{file_ext}"))
            top_right.save(os.path.join(output_folder, f"{file_basename}_2_top_right{file_ext}"))
            bottom_left.save(os.path.join(output_folder, f"{file_basename}_3_bottom_left{file_ext}"))
            bottom_right.save(os.path.join(output_folder, f"{file_basename}_4_bottom_right{file_ext}"))
        display(to_markdown(f"**URL:**{url}\n\n**Filename:** {filename}"))


        # Delete the input file
        os.remove(f"{input_folder}/{filename}")


with open(input_list_image, 'r') as limgurl:
      imgurls = limgurl.readlines()
      for imgurl in imgurls:
        download_image(imgurl)
